<a href="https://colab.research.google.com/github/soyHouston256/CodeJam/blob/master/modulo4_analisis_exploratorio/Outlier_and_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://posgrado.utec.edu.pe/sites/default/files/2023-08/Testimonial-home-2.jpg" alt="HTML5 Icon" width="900" height="250" >

#**Laboratorio: Detección de Outliers en Datos de Taxis de NYC**

---

##**Objetivo general**

Aplicar técnicas de detección de outliers sobre datos reales de viajes en taxi en la ciudad de Nueva York, analizando su fundamento teórico, forma de implementación y capacidad para identificar observaciones atípicas en un entorno multivariado.

## **1. Introducción**

Los valores atípicos (**outliers**) son observaciones que se alejan significativamente del patrón general de los datos. Su detección es crucial en tareas de análisis exploratorio, limpieza, modelado y monitoreo.

Este laboratorio aborda técnicas univariadas y multivariadas para detectar outliers, usando datos simulados basados en el NYC Taxi Dataset.

Se aplicarán cinco métodos:


- Z-score

- IQR (boxplot)

- DBSCAN

- LOF (Local Outlier Factor)

- Isolation Forest

### **Descargar Data**

Este conjunto de datos contiene información detallada sobre millones de viajes en taxi realizados en la ciudad de Nueva York durante el año 2018.

Es un dataset clásico para análisis de movilidad, detección de anomalías, pricing dinámico y aprendizaje automático.

Cada fila representa un viaje individual y las columnas incluyen:

  > **pickup_datetime y dropoff_datetime**: fechas y horas de inicio y fin del viaje.
  >
  > **trip_distance**: distancia total recorrida (en millas).
  >
  > **fare_amount**: monto base de la tarifa del viaje.
  >
  > **tip_amount**: monto de la propina.
  >
  > **payment_type**: tipo de pago (efectivo, tarjeta, etc.).
  >
  > **passenger_coun**: número de pasajeros.
  >
  > **pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude**: coordenadas geográficas del viaje.


**Fuente:** https://www.kaggle.com/datasets/neilclack/nyc-taxi-trip-data-google-public-data?select=original_cleaned_nyc_taxi_data_2018.csv

In [1]:
from google.colab import files
files.upload()  # Sube kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"soyhouston","key":"3e71bf3d6ba0e5bdfa50fc25686d4f5c"}'}

In [2]:
import os
import zipfile

# Crear la carpeta .kaggle
os.makedirs("/root/.kaggle", exist_ok=True)

# Mover el archivo json a la carpeta de configuración
!mv kaggle.json /root/.kaggle/kaggle.json
os.chmod("/root/.kaggle/kaggle.json", 600)


In [3]:
%%capture
!pip3 install -q kagglehub

In [4]:

import kagglehub

# Descargar dataset
path = kagglehub.dataset_download("neilclack/nyc-taxi-trip-data-google-public-data")

print("Ruta local de descarga:", path)

Ruta local de descarga: /kaggle/input/nyc-taxi-trip-data-google-public-data


In [5]:
import os

# Ver archivos descargados
os.listdir(path)


['.nfs000000003277b8f000000515',
 'original_cleaned_nyc_taxi_data_2018.csv',
 'taxi_trip_data.csv',
 'taxi_zone_geo.csv']

In [11]:
import pandas as pd

path = "/kaggle/input/nyc-taxi-trip-data-google-public-data/original_cleaned_nyc_taxi_data_2018.csv"

df = pd.read_csv(path).sample(n=20_000, random_state=42)
df.head()


,Unnamed: 0,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,total_amount,pickup_location_id,dropoff_location_id,year,month,day,day_of_week,hour_of_day,trip_duration,calculated_total_amount
3889775,4144169,7.40,1,Y,2,28.0,0.5,0.5,0.00,0.0,...,29.30,48,82,2018,3,27,1,0,2876.0,35.16
6908713,7472631,9.00,1,N,1,27.5,0.5,0.5,5.75,0.0,...,34.55,261,202,2018,12,3,0,22,2433.0,47.47
4149018,4425107,7.80,1,N,1,23.5,0.5,0.5,3.72,0.0,...,28.52,186,116,2018,10,17,2,3,3469.0,31.60
6816202,7366751,11.78,1,N,1,33.0,0.5,0.5,6.86,0.0,...,41.16,138,64,2018,5,4,4,23,1549.0,30.35
6140876,6614733,8.43,1,N,1,24.5,0.0,0.5,2.00,0.0,...,27.30,13,263,2018,11,12,0,19,2702.0,27.95


In [ ]:
df.shape

(20000, 21)

## **2. Z-score**

### **2.1 Fundamento teórico**


El Z-score indica cuántas desviaciones estándar se encuentra un valor con respecto a la media.

Se define como:


<img src="https://almablog-media.s3.ap-south-1.amazonaws.com/image015_fd507da112.jpg" alt="HTML5 Icon" width="400" height="200" >

Se considera outlier todo valor con ∣𝑍𝑖∣>3, bajo el supuesto de normalidad.

## **2.2 Implementación**

In [ ]:
from scipy.stats import zscore

variables = ['fare_amount', 'trip_distance']
for var in variables:
    df[f"{var}_z"] = zscore(df[var])
    df[f"{var}_outlier_z"] = df[f"{var}_z"].abs() > 3


##**3. IQR (Boxplot)**

### **3.1 Fundamento teórico**

El rango intercuartílico (IQR) se define como:

IQR=Q3−Q1

Donde:

> 𝑄1: percentil 25
>
> 𝑄3: percentil 75






<img src="https://www.simplypsychology.org/wp-content/uploads/boxplot-1.jpg" alt="HTML5 Icon" width="500" height="200" >

Se considera outlier todo valor fuera de:

(Q1−1.5⋅IQR, Q3+1.5⋅IQR)

### **3.2 Implementación**

In [ ]:
def detect_iqr_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    return (series < Q1 - 1.5 * IQR) | (series > Q3 + 1.5 * IQR)

df['iqr_outlier_fare'] = detect_iqr_outliers(df['fare_amount'])


## **4. DBSCAN**

### **4.1 Fundamento teórico**

**DBSCAN** detecta agrupamientos por densidad. Un punto se considera outlier si tiene menos de min_samples vecinos dentro de un radio eps.




<img src="https://mineracaodedados.wordpress.com/wp-content/uploads/2018/02/dbscan.png" alt="HTML5 Icon" width="500" height="300" >



### **4.2 Implementación**

In [ ]:
# Reducimos el tamaño con muestreo aleatorio reproducible
df_sample = df[['trip_distance', 'fare_amount', 'trip_duration']].dropna()

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

X_scaled = StandardScaler().fit_transform(df_sample)

dbscan = DBSCAN(eps=1.2, min_samples=10)
df['dbscan_label'] = dbscan.fit_predict(X_scaled)
df['dbscan_outlier'] = df['dbscan_label'] == -1


## **5. LOF (Local Outlier Factor)**

### **5.1 Fundamento teórico**

LOF compara la densidad local de un punto con la de sus vecinos. Si un punto tiene baja densidad relativa, se considera un outlier.




<img src="https://qu4nt.github.io/sklearn-doc-es/_images/sphx_glr_plot_lof_novelty_detection_001.png" alt="HTML5 Icon" width="450" height="300" >

### **5.2 Implementación**

In [ ]:
df_sample = df[['trip_distance', 'fare_amount', 'trip_duration']].dropna()

from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(df_sample)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
lof = lof.fit(X_scaled)

df['lof_score'] = lof.fit_predict(X_scaled)
df['lof_outlier'] = df['lof_score'] == -1


## **6. Isolation Forest**

### **6.1 Fundamento teórico**

Isolation Forest aísla puntos de forma aleatoria. Los outliers, al estar más aislados, requieren menos divisiones. Eso se traduce en una menor profundidad media del árbol.



<img src="https://media.licdn.com/dms/image/v2/D4D08AQEiI2518F5qhg/croft-frontend-shrinkToFit1024/croft-frontend-shrinkToFit1024/0/1700684114746?e=2147483647&v=beta&t=w-7UygZ_NYGxA30dvc6GbDc5OSQgq1ASAdSukZ8ENFY" alt="HTML5 Icon" width="500" height="280" >

### **6.2 Implementación**

In [ ]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
df['if_outlier'] = iso.fit_predict(X_scaled) == -1


## **7. Análisis Visual y Cuantitativo**

### **7.1 Boxplots para Z-score e IQR**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.boxplot(data=df, x="fare_amount", ax=axes[0])
axes[0].set_title("Fare Amount")
sns.boxplot(data=df, x="trip_distance", ax=axes[1])
axes[1].set_title("Trip Distance")
plt.tight_layout()
plt.show()


In [ ]:
# Excluir outliers antes de graficar
df_no_outliers = df[~df['iqr_outlier_fare']]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.boxplot(data=df_no_outliers, x="fare_amount", ax=axes[0])
axes[0].set_title("Fare Amount - sin outliers")

sns.boxplot(data=df_no_outliers, x="trip_distance", ax=axes[1])
axes[1].set_title("Trip Distance - sin outliers")

plt.tight_layout()
plt.show()


### **7.2 Conteo de outliers por técnica**

In [ ]:
print("Z-score Fare:", df["fare_amount_outlier_z"].sum())
print("Z-score Distance:", df["trip_distance_outlier_z"].sum())
print("IQR Fare:", df["iqr_outlier_fare"].sum())
print("DBSCAN:", df["dbscan_outlier"].sum())
print("LOF:", df["lof_outlier"].sum())
print("Isolation Forest:", df["if_outlier"].sum())


## **8. Visualización de técnicas multivariadas**


### **8.1 DBSCAN**


In [ ]:
sns.scatterplot(data=df, x="trip_distance", y="fare_amount", hue=df["dbscan_label"])
plt.title("DBSCAN: Clústeres y Outliers")
plt.show()


### **8.2 LOF**

In [ ]:
sns.scatterplot(data=df, x="trip_distance", y="fare_amount", hue=df["lof_outlier"])
plt.title("LOF: Outliers detectados")
plt.show()


### **8.3 Isolation Forest**

In [ ]:
sns.scatterplot(data=df, x="trip_distance", y="fare_amount", hue=df["if_outlier"])
plt.title("Isolation Forest: Outliers detectados")
plt.show()


## **9. Comparación y análisis cruzado**

### **9.1 Conteo general**

In [ ]:
outlier_cols = ['fare_amount_outlier_z', 'iqr_outlier_fare', 'dbscan_outlier', 'lof_outlier', 'if_outlier']
df[outlier_cols].sum()


### **9.2 Coincidencias entre métodos**

In [ ]:
df["lof_if"] = df["lof_outlier"] & df["if_outlier"]
df["z_iqr"] = df["fare_amount_outlier_z"] & df["iqr_outlier_fare"]

print("LOF & IF:", df["lof_if"].sum())
print("Z-score & IQR:", df["z_iqr"].sum())


### **9.3 Outliers detectados por múltiples métodos**

In [ ]:
df["outlier_count"] = df[outlier_cols].sum(axis=1)
outliers_multi = df[df["outlier_count"] >= 3]
print("Outliers detectados por 3+ técnicas:", len(outliers_multi))


In [ ]:
outliers_multi.head()

## **10. Conclusión**

Las técnicas univariadas como Z-score e IQR son simples pero limitadas en escenarios multivariados o distribuciones sesgadas. En cambio, técnicas como DBSCAN, LOF e Isolation Forest permiten detectar patrones más complejos, aunque requieren más configuración y análisis visual.

La detección de outliers no debe usarse de forma ciega. Debe combinarse con conocimiento del dominio, validación visual, y análisis de impacto en modelos posteriores. Una observación extrema no siempre es un error: puede ser una oportunidad.

---

## **Trabajo**

**Visualización comparativa de técnicas univariadas:**

Programa una función que grafique lado a lado los boxplots de fare_amount con y sin outliers (según IQR). ¿Cómo cambia la forma de la distribución al remover los outliers? ¿Cuántos valores fueron eliminados?

**Exploración de inconsistencias multivariadas:**

Escribe código para detectar viajes en los que la duración sea muy corta pero la tarifa total muy alta. Usa reglas lógicas personalizadas o una técnica no supervisada como Isolation Forest. ¿Cuántos casos detectas? ¿Qué podría explicarlos?

**Matriz de coincidencias entre técnicas de outliers:**

Crea una matriz de correlación binaria entre los métodos Z-score, IQR, DBSCAN, LOF e Isolation Forest (por columnas booleanas de outliers). ¿Qué técnicas coinciden más? Visualízalo con un heatmap.

**Función generalizada para aplicar DBSCAN:**

Implementa una función parametrizable aplicar_dbscan(df, variables, eps, min_samples) que permita probar distintas configuraciones de eps y min_samples sobre subconjuntos de variables. Evalúa cómo cambia la cantidad de outliers detectados.

**Perfil detallado de un outlier extremo:**

Elige una observación detectada como outlier por al menos 3 técnicas. Programa una función que grafique su posición relativa en cada una de las variables analizadas (fare_amount, trip_distance, tip_amount, etc.) comparada con el resto de los datos.

---

# Gracias por completar este laboratorio!

---
